In [106]:
from docx import Document
import os
import re
import csv
import sys

In [107]:
'''
Author: Yufei

Get the indent space of the paragraph, used in find_patterns and match function

-para = paragraph

return a num means the indent space or null if no indent
'''
def indent(para):
    return para.paragraph_format.first_line_indent

### indent(para) Example
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Change of control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [108]:
'''
Author: Alan

Check if the paragraph is list paragraph rather than normal paragraph, used in find_patterns and match function

-para = paragraph

return True/False
'''
def isListPara(para):
    paraFormat = para.style.name
    if paraFormat == 'List Paragraph':
        return True
    return False

### isListPara(para) Example
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [109]:
'''
Author: Alan

Check if the paragraph is Normal rather than list paragraph，used in match function

-para = paragraph

return True/False
'''
def isNormal(para):
    paraFormat = para.style.name
    if paraFormat == 'Normal':
        return True
    return False

### isNormal(para) Example
any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

In [110]:
'''
Author: Alan

Check if the paragraph followed by list paragraph, used in find_patterns function

-paras = all the paragraph
-curIndex = the index of current paragraph

return True/False
'''
def isNormalFollowingListPara(paras, curIndex):
    targetFormat = paras[curIndex].style.name
    if curIndex == len(paras) - 1:
        return False
    nextFormat = paras[curIndex+1].style.name
    if nextFormat == 'List Paragraph' and targetFormat == 'Normal':
        return True
    return False

### isNormalFollwingListPara(paras, curIndex) Example
Change of control means that 
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [111]:
'''
Author: Zheng

Check if the bold font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def bold(para):
    paras_toPrint = []
    for run in para.runs:
        if run.bold:
            #print run.text,'bold'
            paras_toPrint.append(run.text)              
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### bold(para) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

**Code** shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [112]:
'''
Author: Zheng

Check if the bold font and keyword in the paragraph, used in find_patterns function

-para = paragraph
-keywords = search words

return True/False
'''
def is_bold(para, keywords):
    paras_toPrint = []
    if keyword.lower() in para.text.lower(): 
        for run in para.runs:
            if run.bold:
                #print run.text,'bold'
                paras_toPrint.append(run.text)
                return True
    return False

### is_bold(para, keywords) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [113]:
'''
Author: Zheng

Check if the italic font and keywords in the paragraph, used in match function

-para = paragraph

return True/False
'''
def is_italic(para, keywords):
    paras_toPrint = []
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.italic:
                paras_toPrint.append(run.text)
                return True
    return False

### is_italic(para, keywords) Example
_Change of control_ means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [114]:
'''
Author: Zheng

Check if the italic font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def italic(para):
    paras_toPrint = []
    for run in para.runs:
        if run.italic:
            #print run.text,'italic'
            return True
            paras_toPrint.append(run.text)
    return False

### italic(para) Example
_Code_ shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [115]:
# Zheng
def is_underline(para, keywords):
    paras_toPrint = []    
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.underline:
                #print run.text,'underline'
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_underline(para, keywords) Example
Change of control(Underline) means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [116]:
'''
Author: Zheng

Check if the underline in the paragraph, used in match function

-para = paragraph

return True/False
'''
def underline(para):
    paras_toPrint = []
    for run in para.runs:
        if run.underline:
            return True
            #print run.text,'underline'
            paras_toPrint.append(run.text)
    return False

### underline(para) Example

Code(Underline) shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [117]:
'''
Author: Zoe

Check if the keyword is double quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_double_quotes(para, keywords):
    words = re.findall(r'\"(.+?)\"', para.text)
    print 'IS Double Quotes', words
    if not words:
        return False
    if para.text.startswith("\"" + words[0] + "\""):   #and words[0].lower() == keyword.lower():
        return True       
    else:
        return False

### is_double_quotes(para, keywords) Example:
"Change of control" means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [118]:
'''
Author: Zoe

Check if the double quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def double_quotes(para):
    """
        Find if paragraph starts with double quotes
        
        return: True or False
    """
    words = re.findall(r'(\".+?\")', para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True       
    else:
        return False

### double_quotes(para) Example:

"Code" shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [119]:
'''
Author: Zoe

Check if the keyword is single quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_single_quotes(para, keywords):
    words = re.findall(r'\'(.+?)\'\s',para.text)
    if not words:
        return False
    if para.text.startswith("\'"+ words[0] + "\'"): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### is_single_quotes(para, keywords) Example:
'Change of control' means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [120]:
'''
Author: Zoe

Check if single quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def single_quotes(para):
    words = re.findall(r'(\'.+?\')\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

### single_quotes(para) Example:

'Code' shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [121]:
'''
Author: Zoe

Check if the keyword is upper case in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_upper_case(para, keywords):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### is_upper_case(para, keywords) Example:
CHANGE OF CONTROL shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [122]:
'''
Author: Zoe

Check if upper case word in the paragraph, used in match function

-para = paragraph

return True/False
'''
def upper_case(para):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

### upper_case(para) Example:

UPPERCASE shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [123]:
'''
Author: Zoe

Remove the stopwords such as articles, prep and so on from keywords

-keywords = search word

return the string without stopwords
'''
from nltk.corpus import stopwords
def remove_stopwords(keywords):
    
   
    
    word_list = re.split(r'\s',keywords)
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    words = " ".join(filtered_words)
    return words

#### Remove_Stopwords Test

In [124]:
keyword = "Change of Control"
print remove_stopwords(keyword)

Change Control


In [125]:
'''
Author: Zoe

Still Testing
'''
def upper_camel_case(para):
    words_list = re.split(" ", para.text)
    new_para = remove_stopwords(para.text)
    new_para_list = re.split(" ", new_para)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_para)
    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_para_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    if len(result) > 1:
        return True
    else:
        return False

### upper_camel_case(para, keywords) Example:
Capitalized Lease Obligations means, with respect to any Person, all outstanding obligations of such Person in respect of Capital Leases, taken at the capitalized amount thereof accounted for as indebtedness in accordance with GAAP.

Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [126]:
'''
Author: Zoe

Still Testing
'''
def is_upper_camel_case(paragraph, keyword):
    words_list = re.compile('\w+').findall(paragraph.text)
    new_paragraph = remove_stopwords(paragraph.text)
    new_paragraph_list = re.compile('\w+').findall(new_paragraph)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_paragraph)

    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_paragraph_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    results = " ".join(result)
    if results.lower() == keyword.lower():
        return True
    else:
        return False

In [127]:
# s = is_upper_camel_case('''"Change of Control" means Other Connection Taxes that are imposed on or measured by net income (however denominated) or that are franchise Taxes or branch profits Taxes.
# ''', 'change of control')
# print s

### is_upper_camel_case() Example:
Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [128]:
'''
Author: Yufei

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search(paras, keywords):
    index = []
    i = 0
    while (i<len(paras)):
        p = paras[i]
        if keywords.lower() in p.text.lower():
            index.append(i)
        i = i+1
    return index

In [129]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-index = paragraph index
-keyword = search words

return True/False list, 
[bold, italic, underline, double_quotes, single_quotes, upper case, (upper camel case), indent space, list paragraph, normal following list paragraph]

'''
def find_patterns(paras, index, keyword):
    paragraph = paras[index]
    patterns = []
    patterns.append(is_bold(paragraph, keyword))
    patterns.append(is_italic(paragraph, keyword))
    patterns.append(is_underline(paragraph, keyword))
    patterns.append(is_double_quotes(paragraph, keyword))
    patterns.append(is_single_quotes(paragraph, keyword))
    patterns.append(is_upper_case(paragraph, keyword))
    #patterns.append(is_upper_camel_case(paragraph, keyword))
    patterns.append(indent(paragraph))
    patterns.append(isListPara(paragraph))
    patterns.append(isNormalFollowingListPara(paras, index))
    return patterns

In [135]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-targetPatterns = the target pattern
-startIndex = paragraph index

return content
'''
def match(paras, targetPatterns, startIndex):
    
    i = startIndex + 1
    match = []
    
    print ('Target  Patterns: ', targetPatterns)
    
    while (i<len(paras)):
        patterns = []
        p = paras[i]
        print('Next Para: ', p.text)

        patterns.append(bold(p))
        patterns.append(italic(p))
        patterns.append(underline(p))
        patterns.append(double_quotes(p))
        patterns.append(single_quotes(p))
        patterns.append(upper_case(p))
        #patterns.append(upper_camel_case(p))
        patterns.append(indent(p))
        patterns.append(isListPara(p))
        patterns.append(isNormalFollowingListPara(paras, i))
        
        print('Next Para Patterns: ', patterns)
        
        if patterns == targetPatterns:
            print 'Bingo Bingo Bingo Bingo'
            break
        i = i + 1
    for x in range(startIndex, i):
        match.append(paras[x].text)
        
    return match

In [136]:
# Abhay
'''
Open the output file in append mode and pull related language from the files in the input folder
'''

########################################################################
out_filepath = 'Output/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Change of Control'] # Column names in output file 

with open(out_path, 'wb') as f:
    wr = csv.writer(f)
    wr.writerow(header)

with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = 'input' 
    
    # Parse through each file in the input folder path
    for fileName in os.listdir(directory_path):
        print fileName
        
        # Create a Document object of each of the files
        #document = Document(directory_path + "/" +fileName)
        document = Document('input/upper_camel_case+underline+after_list_paragraph.docx')
        
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        keyword = "Change of Control"
        keywordindex = search(paras, keyword)
        
        print('keywordIndex:', keywordindex)
        
        results = []
        result = ""
        for index in keywordindex:
            print 'Target Para', paras[index].text
            target = find_patterns(paras, index, keyword)
            
            if index == paras_number - 1:
                result = paras[index]
            else:
                result = match(paras, target, index)
            
            results.append(result)

        wr = csv.writer(f)
        wr.writerow([fileName, keyword, results])
        
print("end")
f.close()

.DS_Store
('keywordIndex:', [0, 3])
Target Para Change of Control means Other Connection Taxes that are imposed on or measured by net income (however denominated) or that are franchise Taxes or branch profits Taxes.
IS Double Quotes []
('Target  Patterns: ', [False, False, True, False, False, False, 266700, False, False])
('Next Para: ', u'Contractual Obligation means, as to any Person, any provision of any security issued by such Person or of any agreement, instrument or other undertaking to which such Person is a party or by which it or any of its property is bound.')
('Next Para Patterns: ', [False, False, True, False, False, False, 266700, False, False])
Bingo Bingo Bingo Bingo
Target Para “Change of Control” means an event or series of events by which:
IS Double Quotes []
('Target  Patterns: ', [False, False, True, False, False, False, None, False, True])
('Next Para: ', u'any \u201cperson\u201d or \u201cgroup\u201d (as such terms are used in Sections 13(d) and 14(d) of the Exchan

In [134]:
os.getcwd()

'/Users/shenglan/Documents/Capstone/CodeV1/Capstone-Project-Mylan'